<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/AGC27_2_1_HLC_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#License: GPL 
#Creator: Mike Smith https://github.com/gomlfx 
#designed for colab/kaggle/Jupyter

In [2]:
#!pip install autogluon
#!pip install --upgrade mxnet

In [3]:
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing

#autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
#metatrader5 file formatv'
df=pd.read_table('/content/USDJPY_Daily_197101040000_202205130000.csv')

In [5]:
print(df.head())
print(df.tail())
df.info()

       <DATE>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  <TICKVOL>  <VOL>  <SPREAD>
0  1971.01.04  357.73  357.73  357.73   357.73          1      0        50
1  1971.01.05  357.81  357.81  357.81   357.81          1      0        50
2  1971.01.06  357.86  357.86  357.86   357.86          1      0        50
3  1971.01.07  357.87  357.87  357.87   357.87          1      0        50
4  1971.01.08  357.82  357.82  357.82   357.82          1      0        50
           <DATE>   <OPEN>   <HIGH>    <LOW>  <CLOSE>  <TICKVOL>  <VOL>  \
13232  2022.05.09  130.491  131.348  130.112  130.319     151730      0   
13233  2022.05.10  130.316  130.570  129.797  130.414     159942      0   
13234  2022.05.11  130.416  130.811  129.443  129.967     167000      0   
13235  2022.05.12  129.935  130.049  127.520  128.303     201610      0   
13236  2022.05.13  128.303  128.745  128.227  128.714      15421      0   

       <SPREAD>  
13232         0  
13233         0  
13234         0  
13235         0  
13236    

In [6]:
df

,<DATE>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>
0,1971.01.04,357.730,357.730,357.730,357.730,1,0,50
1,1971.01.05,357.810,357.810,357.810,357.810,1,0,50
2,1971.01.06,357.860,357.860,357.860,357.860,1,0,50
3,1971.01.07,357.870,357.870,357.870,357.870,1,0,50
4,1971.01.08,357.820,357.820,357.820,357.820,1,0,50
...,...,...,...,...,...,...,...,...
13232,2022.05.09,130.491,131.348,130.112,130.319,151730,0,0
13233,2022.05.10,130.316,130.570,129.797,130.414,159942,0,0
13234,2022.05.11,130.416,130.811,129.443,129.967,167000,0,0
13235,2022.05.12,129.935,130.049,127.520,128.303,201610,0,0


In [7]:
d = df[["<OPEN>","<HIGH>","<LOW>","<CLOSE>"]]
df = pd.DataFrame(data=d)
df

,<OPEN>,<HIGH>,<LOW>,<CLOSE>
0,357.730,357.730,357.730,357.730
1,357.810,357.810,357.810,357.810
2,357.860,357.860,357.860,357.860
3,357.870,357.870,357.870,357.870
4,357.820,357.820,357.820,357.820
...,...,...,...,...
13232,130.491,131.348,130.112,130.319
13233,130.316,130.570,129.797,130.414
13234,130.416,130.811,129.443,129.967
13235,129.935,130.049,127.520,128.303


In [8]:
df_raw1 = pd.DataFrame(data=df)
df_raw1

,<OPEN>,<HIGH>,<LOW>,<CLOSE>
0,357.730,357.730,357.730,357.730
1,357.810,357.810,357.810,357.810
2,357.860,357.860,357.860,357.860
3,357.870,357.870,357.870,357.870
4,357.820,357.820,357.820,357.820
...,...,...,...,...
13232,130.491,131.348,130.112,130.319
13233,130.316,130.570,129.797,130.414
13234,130.416,130.811,129.443,129.967
13235,129.935,130.049,127.520,128.303


In [9]:
stop_df = pd.DataFrame(data=df.tail(40))  
stop_df.iloc[:-1] 
stop_df1 = stop_df[stop_df['<OPEN>'] > stop_df['<CLOSE>']]
print(stop_df1)

stop_df2 = stop_df[stop_df['<OPEN>'] < stop_df['<CLOSE>']]
print(stop_df2)


        <OPEN>   <HIGH>    <LOW>  <CLOSE>
13201  122.352  122.437  121.178  122.089
13203  123.863  124.302  121.976  122.868
13204  122.848  123.205  121.309  121.831
13205  121.836  122.450  121.281  121.695
13219  128.871  129.400  127.458  127.882
13222  128.553  128.866  127.521  128.127
13223  128.124  128.220  127.023  127.217
13226  130.795  130.944  129.315  129.863
13228  130.163  130.292  129.698  130.131
13229  130.125  130.390  128.623  129.081
13232  130.491  131.348  130.112  130.319
13234  130.416  130.811  129.443  129.967
13235  129.935  130.049  127.520  128.303
        <OPEN>   <HIGH>    <LOW>  <CLOSE>
13197  119.102  119.499  119.081  119.461
13198  119.457  121.027  119.432  120.826
13199  120.818  121.412  120.592  121.120
13200  121.121  122.408  120.950  122.370
13202  122.004  125.100  122.004  123.888
13206  121.669  123.036  121.601  122.559
13207  122.597  122.959  122.261  122.786
13208  122.786  123.667  122.368  123.599
13209  123.579  124.050  123.455  

In [10]:

stop_df1['stop_for_sell'] = stop_df1['<HIGH>'] - stop_df1['<OPEN>']
print(stop_df1.stop_for_sell)

stop_df2['stop_for_buy'] = stop_df2['<OPEN>'] - stop_df2['<LOW>']
print(stop_df2.stop_for_buy)

13201    0.085
13203    0.439
13204    0.357
13205    0.614
13219    0.529
13222    0.313
13223    0.096
13226    0.149
13228    0.129
13229    0.265
13232    0.857
13234    0.395
13235    0.114
Name: stop_for_sell, dtype: float64
13197    0.021
13198    0.025
13199    0.226
13200    0.171
13202    0.000
13206    0.068
13207    0.336
13208    0.418
13209    0.124
13210    0.289
13211    0.277
13212    0.000
13213    0.581
13214    0.027
13215    0.570
13216    0.009
13217    0.081
13218    0.023
13220    0.080
13221    0.615
13224    0.269
13225    0.062
13227    0.114
13230    0.302
13231    0.091
13233    0.519
13236    0.076
Name: stop_for_buy, dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
stop_for_sell = stop_df1.stop_for_sell.describe(percentiles=[.25,.5,.75,.9,.95,.99])
stop_for_buy = stop_df2.stop_for_buy.describe(percentiles=[.25,.5,.75,.9,.95,.99])
 

In [12]:
print(stop_for_sell)
print(stop_for_buy)

count    13.000000
mean      0.334000
std       0.233403
min       0.085000
25%       0.129000
50%       0.313000
75%       0.439000
90%       0.597000
95%       0.711200
99%       0.827840
max       0.857000
Name: stop_for_sell, dtype: float64
count    27.000000
mean      0.199037
std       0.195963
min       0.000000
25%       0.044500
50%       0.114000
75%       0.295500
90%       0.539400
95%       0.577700
99%       0.606160
max       0.615000
Name: stop_for_buy, dtype: float64


In [13]:
df.rename({'<OPEN>':'p_o','<HIGH>':'p_h','<LOW>':'p_l','<CLOSE>':'p_c'}, axis='columns', inplace=True)

In [14]:
df.info

<bound method DataFrame.info of            p_o      p_h      p_l      p_c
0      357.730  357.730  357.730  357.730
1      357.810  357.810  357.810  357.810
2      357.860  357.860  357.860  357.860
3      357.870  357.870  357.870  357.870
4      357.820  357.820  357.820  357.820
...        ...      ...      ...      ...
13232  130.491  131.348  130.112  130.319
13233  130.316  130.570  129.797  130.414
13234  130.416  130.811  129.443  129.967
13235  129.935  130.049  127.520  128.303
13236  128.303  128.745  128.227  128.714

[13237 rows x 4 columns]>

In [15]:
print(df.columns)

Index(['p_o', 'p_h', 'p_l', 'p_c'], dtype='object')


In [16]:
#Features

#230 periods ago
df['p_Open_last_period230'] = df['p_o'].shift(230) 
df['p_High_last_period230'] = df['p_h'].shift(230)
df['p_Low_last_period230'] = df['p_l'].shift(230)
df['p_Close_last_period230'] = df['p_c'].shift(230)
#229 periods ago
df['p_Open_last_period229'] = df['p_o'].shift(229) 
df['p_High_last_period229'] = df['p_h'].shift(229)
df['p_Low_last_period229'] = df['p_l'].shift(229)
df['p_Close_last_period229'] = df['p_c'].shift(229)
#228 periods ago
df['p_Open_last_period228'] = df['p_o'].shift(228) 
df['p_High_last_period228'] = df['p_h'].shift(228)
df['p_Low_last_period228'] = df['p_l'].shift(228)
df['p_Close_last_period228'] = df['p_c'].shift(228)
#227 periods ago
df['p_Open_last_period227'] = df['p_o'].shift(227) 
df['p_High_last_period227'] = df['p_h'].shift(227)
df['p_Low_last_period227'] = df['p_l'].shift(227)
df['p_Close_last_period227'] = df['p_c'].shift(227)
#226 periods ago
df['p_Open_last_period226'] = df['p_o'].shift(226) 
df['p_High_last_period226'] = df['p_h'].shift(226)
df['p_Low_last_period226'] = df['p_l'].shift(226)
df['p_Close_last_period226'] = df['p_c'].shift(226)
#225 periods ago
df['p_Open_last_period225'] = df['p_o'].shift(225) 
df['p_High_last_period225'] = df['p_h'].shift(225)
df['p_Low_last_period225'] = df['p_l'].shift(225)
df['p_Close_last_period225'] = df['p_c'].shift(225)
#224 periods ago
df['p_Open_last_period224'] = df['p_o'].shift(224) 
df['p_High_last_period224'] = df['p_h'].shift(224)
df['p_Low_last_period224'] = df['p_l'].shift(224)
df['p_Close_last_period224'] = df['p_c'].shift(224)
#223 periods ago
df['p_Open_last_period223'] = df['p_o'].shift(223) 
df['p_High_last_period223'] = df['p_h'].shift(223)
df['p_Low_last_period223'] = df['p_l'].shift(223)
df['p_Close_last_period223'] = df['p_c'].shift(223)
#222 periods ago
df['p_Open_last_period222'] = df['p_o'].shift(222) 
df['p_High_last_period222'] = df['p_h'].shift(222)
df['p_Low_last_period222'] = df['p_l'].shift(222)
df['p_Close_last_period222'] = df['p_c'].shift(222)
#221 periods ago
df['p_Open_last_period221'] = df['p_o'].shift(221) 
df['p_High_last_period221'] = df['p_h'].shift(221)
df['p_Low_last_period221'] = df['p_l'].shift(221)
df['p_Close_last_period221'] = df['p_c'].shift(221)
#220 periods ago
df['p_Open_last_period220'] = df['p_o'].shift(220) 
df['p_High_last_period220'] = df['p_h'].shift(220)
df['p_Low_last_period220'] = df['p_l'].shift(220)
df['p_Close_last_period220'] = df['p_c'].shift(220)
#219 periods ago
df['p_Open_last_period219'] = df['p_o'].shift(219) 
df['p_High_last_period219'] = df['p_h'].shift(219)
df['p_Low_last_period219'] = df['p_l'].shift(219)
df['p_Close_last_period219'] = df['p_c'].shift(219)
#218 periods ago
df['p_Open_last_period218'] = df['p_o'].shift(218) 
df['p_High_last_period218'] = df['p_h'].shift(218)
df['p_Low_last_period218'] = df['p_l'].shift(218)
df['p_Close_last_period218'] = df['p_c'].shift(218)
#217 periods ago
df['p_Open_last_period217'] = df['p_o'].shift(217) 
df['p_High_last_period217'] = df['p_h'].shift(217)
df['p_Low_last_period217'] = df['p_l'].shift(217)
df['p_Close_last_period217'] = df['p_c'].shift(217)
#216 periods ago
df['p_Open_last_period216'] = df['p_o'].shift(216) 
df['p_High_last_period216'] = df['p_h'].shift(216)
df['p_Low_last_period216'] = df['p_l'].shift(216)
df['p_Close_last_period216'] = df['p_c'].shift(216)
#215 periods ago
df['p_Open_last_period215'] = df['p_o'].shift(215) 
df['p_High_last_period215'] = df['p_h'].shift(215)
df['p_Low_last_period215'] = df['p_l'].shift(215)
df['p_Close_last_period215'] = df['p_c'].shift(215)
#214 periods ago
df['p_Open_last_period214'] = df['p_o'].shift(214) 
df['p_High_last_period214'] = df['p_h'].shift(214)
df['p_Low_last_period214'] = df['p_l'].shift(214)
df['p_Close_last_period214'] = df['p_c'].shift(214)
#214 periods ago
df['p_Open_last_period214'] = df['p_o'].shift(214) 
df['p_High_last_period214'] = df['p_h'].shift(214)
df['p_Low_last_period214'] = df['p_l'].shift(214)
df['p_Close_last_period214'] = df['p_c'].shift(214)
#213 periods ago
df['p_Open_last_period213'] = df['p_o'].shift(213) 
df['p_High_last_period213'] = df['p_h'].shift(213)
df['p_Low_last_period213'] = df['p_l'].shift(213)
df['p_Close_last_period213'] = df['p_c'].shift(213)
#212 periods ago
df['p_Open_last_period212'] = df['p_o'].shift(212) 
df['p_High_last_period212'] = df['p_h'].shift(212)
df['p_Low_last_period212'] = df['p_l'].shift(212)
df['p_Close_last_period212'] = df['p_c'].shift(212)
#211 periods ago
df['p_Open_last_period211'] = df['p_o'].shift(211) 
df['p_High_last_period211'] = df['p_h'].shift(211)
df['p_Low_last_period211'] = df['p_l'].shift(211)
df['p_Close_last_period211'] = df['p_c'].shift(211)
#210 periods ago
df['p_Open_last_period210'] = df['p_o'].shift(210) 
df['p_High_last_period210'] = df['p_h'].shift(210)
df['p_Low_last_period210'] = df['p_l'].shift(210)
df['p_Close_last_period210'] = df['p_c'].shift(210)
#209 periods ago
df['p_Open_last_period209'] = df['p_o'].shift(209) 
df['p_High_last_period209'] = df['p_h'].shift(209)
df['p_Low_last_period209'] = df['p_l'].shift(209)
df['p_Close_last_period209'] = df['p_c'].shift(209)
#208 periods ago
df['p_Open_last_period208'] = df['p_o'].shift(208) 
df['p_High_last_period208'] = df['p_h'].shift(208)
df['p_Low_last_period208'] = df['p_l'].shift(208)
df['p_Close_last_period208'] = df['p_c'].shift(208)
#207 periods ago
df['p_Open_last_period207'] = df['p_o'].shift(207) 
df['p_High_last_period207'] = df['p_h'].shift(207)
df['p_Low_last_period207'] = df['p_l'].shift(207)
df['p_Close_last_period207'] = df['p_c'].shift(207)
#206 periods ago
df['p_Open_last_period206'] = df['p_o'].shift(206) 
df['p_High_last_period206'] = df['p_h'].shift(206)
df['p_Low_last_period206'] = df['p_l'].shift(206)
df['p_Close_last_period206'] = df['p_c'].shift(206)
#205 periods ago
df['p_Open_last_period205'] = df['p_o'].shift(205) 
df['p_High_last_period205'] = df['p_h'].shift(205)
df['p_Low_last_period205'] = df['p_l'].shift(205)
df['p_Close_last_period205'] = df['p_c'].shift(205)
#204 periods ago
df['p_Open_last_period204'] = df['p_o'].shift(204) 
df['p_High_last_period204'] = df['p_h'].shift(204)
df['p_Low_last_period204'] = df['p_l'].shift(204)
df['p_Close_last_period204'] = df['p_c'].shift(204)
#203 periods ago
df['p_Open_last_period203'] = df['p_o'].shift(203) 
df['p_High_last_period203'] = df['p_h'].shift(203)
df['p_Low_last_period203'] = df['p_l'].shift(203)
df['p_Close_last_period203'] = df['p_c'].shift(203)
#202 periods ago
df['p_Open_last_period202'] = df['p_o'].shift(202) 
df['p_High_last_period202'] = df['p_h'].shift(202)
df['p_Low_last_period202'] = df['p_l'].shift(202)
df['p_Close_last_period202'] = df['p_c'].shift(202)
#201 periods ago
df['p_Open_last_period201'] = df['p_o'].shift(201) 
df['p_High_last_period201'] = df['p_h'].shift(201)
df['p_Low_last_period201'] = df['p_l'].shift(201)
df['p_Close_last_period201'] = df['p_c'].shift(201)
#200 periods ago
df['p_Open_last_period200'] = df['p_o'].shift(200) 
df['p_High_last_period200'] = df['p_h'].shift(200)
df['p_Low_last_period200'] = df['p_l'].shift(200)
df['p_Close_last_period200'] = df['p_c'].shift(200)
#199 periods ago
df['p_Open_last_period199'] = df['p_o'].shift(199) 
df['p_High_last_period199'] = df['p_h'].shift(199)
df['p_Low_last_period199'] = df['p_l'].shift(199)
df['p_Close_last_period199'] = df['p_c'].shift(199)
#198 periods ago
df['p_Open_last_period198'] = df['p_o'].shift(198) 
df['p_High_last_period198'] = df['p_h'].shift(198)
df['p_Low_last_period198'] = df['p_l'].shift(198)
df['p_Close_last_period198'] = df['p_c'].shift(198)
#197 periods ago
df['p_Open_last_period197'] = df['p_o'].shift(197) 
df['p_High_last_period197'] = df['p_h'].shift(197)
df['p_Low_last_period197'] = df['p_l'].shift(197)
df['p_Close_last_period197'] = df['p_c'].shift(197)
#196 periods ago
df['p_Open_last_period196'] = df['p_o'].shift(196) 
df['p_High_last_period196'] = df['p_h'].shift(196)
df['p_Low_last_period196'] = df['p_l'].shift(196)
df['p_Close_last_period196'] = df['p_c'].shift(196)
#195 periods ago
df['p_Open_last_period195'] = df['p_o'].shift(195) 
df['p_High_last_period195'] = df['p_h'].shift(195)
df['p_Low_last_period195'] = df['p_l'].shift(195)
df['p_Close_last_period195'] = df['p_c'].shift(195)
#194 periods ago
df['p_Open_last_period194'] = df['p_o'].shift(194) 
df['p_High_last_period194'] = df['p_h'].shift(194)
df['p_Low_last_period194'] = df['p_l'].shift(194)
df['p_Close_last_period194'] = df['p_c'].shift(194)
#193 periods ago
df['p_Open_last_period193'] = df['p_o'].shift(193) 
df['p_High_last_period193'] = df['p_h'].shift(193)
df['p_Low_last_period193'] = df['p_l'].shift(193)
df['p_Close_last_period193'] = df['p_c'].shift(193)
#192 periods ago
df['p_Open_last_period192'] = df['p_o'].shift(192) 
df['p_High_last_period192'] = df['p_h'].shift(192)
df['p_Low_last_period192'] = df['p_l'].shift(192)
df['p_Close_last_period192'] = df['p_c'].shift(192)
#191 periods ago
df['p_Open_last_period191'] = df['p_o'].shift(191) 
df['p_High_last_period191'] = df['p_h'].shift(191)
df['p_Low_last_period191'] = df['p_l'].shift(191)
df['p_Close_last_period191'] = df['p_c'].shift(191)
#190 periods ago
df['p_Open_last_period190'] = df['p_o'].shift(190) 
df['p_High_last_period190'] = df['p_h'].shift(190)
df['p_Low_last_period190'] = df['p_l'].shift(190)
df['p_Close_last_period190'] = df['p_c'].shift(190)
#189 periods ago
df['p_Open_last_period189'] = df['p_o'].shift(189) 
df['p_High_last_period189'] = df['p_h'].shift(189)
df['p_Low_last_period189'] = df['p_l'].shift(189)
df['p_Close_last_period189'] = df['p_c'].shift(189)
#188 periods ago
df['p_Open_last_period188'] = df['p_o'].shift(188) 
df['p_High_last_period188'] = df['p_h'].shift(188)
df['p_Low_last_period188'] = df['p_l'].shift(188)
df['p_Close_last_period188'] = df['p_c'].shift(188)
#187 periods ago
df['p_Open_last_period187'] = df['p_o'].shift(187) 
df['p_High_last_period187'] = df['p_h'].shift(187)
df['p_Low_last_period187'] = df['p_l'].shift(187)
df['p_Close_last_period187'] = df['p_c'].shift(187)
#186 periods ago
df['p_Open_last_period186'] = df['p_o'].shift(186) 
df['p_High_last_period186'] = df['p_h'].shift(186)
df['p_Low_last_period186'] = df['p_l'].shift(186)
df['p_Close_last_period186'] = df['p_c'].shift(186)
#185 periods ago
df['p_Open_last_period185'] = df['p_o'].shift(185) 
df['p_High_last_period185'] = df['p_h'].shift(185)
df['p_Low_last_period185'] = df['p_l'].shift(185)
df['p_Close_last_period185'] = df['p_c'].shift(185)
#184 periods ago
df['p_Open_last_period184'] = df['p_o'].shift(184) 
df['p_High_last_period184'] = df['p_h'].shift(184)
df['p_Low_last_period184'] = df['p_l'].shift(184)
df['p_Close_last_period184'] = df['p_c'].shift(184)
#183 periods ago
df['p_Open_last_period183'] = df['p_o'].shift(183) 
df['p_High_last_period183'] = df['p_h'].shift(183)
df['p_Low_last_period183'] = df['p_l'].shift(183)
df['p_Close_last_period183'] = df['p_c'].shift(183)
#182 periods ago
df['p_Open_last_period182'] = df['p_o'].shift(182) 
df['p_High_last_period182'] = df['p_h'].shift(182)
df['p_Low_last_period182'] = df['p_l'].shift(182)
df['p_Close_last_period182'] = df['p_c'].shift(182)
#181 periods ago
df['p_Open_last_period181'] = df['p_o'].shift(181) 
df['p_High_last_period181'] = df['p_h'].shift(181)
df['p_Low_last_period181'] = df['p_l'].shift(181)
df['p_Close_last_period181'] = df['p_c'].shift(181)
#180 periods ago
df['p_Open_last_period180'] = df['p_o'].shift(180) 
df['p_High_last_period180'] = df['p_h'].shift(180)
df['p_Low_last_period180'] = df['p_l'].shift(180)
df['p_Close_last_period180'] = df['p_c'].shift(180)
#179 periods ago
df['p_Open_last_period179'] = df['p_o'].shift(179) 
df['p_High_last_period179'] = df['p_h'].shift(179)
df['p_Low_last_period179'] = df['p_l'].shift(179)
df['p_Close_last_period179'] = df['p_c'].shift(179)
#178 periods ago
df['p_Open_last_period178'] = df['p_o'].shift(178) 
df['p_High_last_period178'] = df['p_h'].shift(178)
df['p_Low_last_period178'] = df['p_l'].shift(178)
df['p_Close_last_period178'] = df['p_c'].shift(178)
#177 periods ago
df['p_Open_last_period177'] = df['p_o'].shift(177) 
df['p_High_last_period177'] = df['p_h'].shift(177)
df['p_Low_last_period177'] = df['p_l'].shift(177)
df['p_Close_last_period177'] = df['p_c'].shift(177)
#176 periods ago
df['p_Open_last_period176'] = df['p_o'].shift(176) 
df['p_High_last_period176'] = df['p_h'].shift(176)
df['p_Low_last_period176'] = df['p_l'].shift(176)
df['p_Close_last_period176'] = df['p_c'].shift(176)
#194 periods ago
df['p_Open_last_period175'] = df['p_o'].shift(175) 
df['p_High_last_period175'] = df['p_h'].shift(175)
df['p_Low_last_period175'] = df['p_l'].shift(175)
df['p_Close_last_period175'] = df['p_c'].shift(175)
#174 periods ago
df['p_Open_last_period174'] = df['p_o'].shift(174) 
df['p_High_last_period174'] = df['p_h'].shift(174)
df['p_Low_last_period174'] = df['p_l'].shift(174)
df['p_Close_last_period174'] = df['p_c'].shift(174)
#173 periods ago
df['p_Open_last_period173'] = df['p_o'].shift(173) 
df['p_High_last_period173'] = df['p_h'].shift(173)
df['p_Low_last_period173'] = df['p_l'].shift(173)
df['p_Close_last_period173'] = df['p_c'].shift(173)
#172 periods ago
df['p_Open_last_period172'] = df['p_o'].shift(172) 
df['p_High_last_period172'] = df['p_h'].shift(172)
df['p_Low_last_period172'] = df['p_l'].shift(172)
df['p_Close_last_period172'] = df['p_c'].shift(172)
#171 periods ago
df['p_Open_last_period171'] = df['p_o'].shift(171) 
df['p_High_last_period171'] = df['p_h'].shift(171)
df['p_Low_last_period171'] = df['p_l'].shift(171)
df['p_Close_last_period171'] = df['p_c'].shift(171)
#170 periods ago
df['p_Open_last_period170'] = df['p_o'].shift(170) 
df['p_High_last_period170'] = df['p_h'].shift(170)
df['p_Low_last_period170'] = df['p_l'].shift(170)
df['p_Close_last_period170'] = df['p_c'].shift(170)
#169 periods ago
df['p_Open_last_period169'] = df['p_o'].shift(169) 
df['p_High_last_period169'] = df['p_h'].shift(169)
df['p_Low_last_period169'] = df['p_l'].shift(169)
df['p_Close_last_period169'] = df['p_c'].shift(169)
#168 periods ago
df['p_Open_last_period168'] = df['p_o'].shift(168) 
df['p_High_last_period168'] = df['p_h'].shift(168)
df['p_Low_last_period168'] = df['p_l'].shift(168)
df['p_Close_last_period168'] = df['p_c'].shift(168)
#167 periods ago
df['p_Open_last_period167'] = df['p_o'].shift(167) 
df['p_High_last_period167'] = df['p_h'].shift(167)
df['p_Low_last_period167'] = df['p_l'].shift(167)
df['p_Close_last_period167'] = df['p_c'].shift(167)
#166 periods ago
df['p_Open_last_period166'] = df['p_o'].shift(166) 
df['p_High_last_period166'] = df['p_h'].shift(166)
df['p_Low_last_period166'] = df['p_l'].shift(166)
df['p_Close_last_period166'] = df['p_c'].shift(166)
#165 periods ago
df['p_Open_last_period165'] = df['p_o'].shift(165) 
df['p_High_last_period165'] = df['p_h'].shift(165)
df['p_Low_last_period165'] = df['p_l'].shift(165)
df['p_Close_last_period165'] = df['p_c'].shift(165)
#164 periods ago
df['p_Open_last_period164'] = df['p_o'].shift(164) 
df['p_High_last_period164'] = df['p_h'].shift(164)
df['p_Low_last_period164'] = df['p_l'].shift(164)
df['p_Close_last_period164'] = df['p_c'].shift(164)
#163 periods ago
df['p_Open_last_period163'] = df['p_o'].shift(163) 
df['p_High_last_period163'] = df['p_h'].shift(163)
df['p_Low_last_period163'] = df['p_l'].shift(163)
df['p_Close_last_period163'] = df['p_c'].shift(163)
#162 periods ago
df['p_Open_last_period162'] = df['p_o'].shift(162) 
df['p_High_last_period162'] = df['p_h'].shift(162)
df['p_Low_last_period162'] = df['p_l'].shift(162)
df['p_Close_last_period162'] = df['p_c'].shift(162)
#161 periods ago
df['p_Open_last_period161'] = df['p_o'].shift(161) 
df['p_High_last_period161'] = df['p_h'].shift(161)
df['p_Low_last_period161'] = df['p_l'].shift(161)
df['p_Close_last_period161'] = df['p_c'].shift(161)
#160 periods ago
df['p_Open_last_period160'] = df['p_o'].shift(160) 
df['p_High_last_period160'] = df['p_h'].shift(160)
df['p_Low_last_period160'] = df['p_l'].shift(160)
df['p_Close_last_period160'] = df['p_c'].shift(160)
#59 periods ago
df['p_Open_last_period159'] = df['p_o'].shift(159) 
df['p_High_last_period159'] = df['p_h'].shift(159)
df['p_Low_last_period159'] = df['p_l'].shift(159)
df['p_Close_last_period159'] = df['p_c'].shift(159)
#158 periods ago
df['p_Open_last_period158'] = df['p_o'].shift(158) 
df['p_High_last_period158'] = df['p_h'].shift(158)
df['p_Low_last_period158'] = df['p_l'].shift(158)
df['p_Close_last_period158'] = df['p_c'].shift(158)
#157 periods ago
df['p_Open_last_period157'] = df['p_o'].shift(157) 
df['p_High_last_period157'] = df['p_h'].shift(157)
df['p_Low_last_period157'] = df['p_l'].shift(157)
df['p_Close_last_period157'] = df['p_c'].shift(157)
#156 periods ago
df['p_Open_last_period156'] = df['p_o'].shift(156) 
df['p_High_last_period156'] = df['p_h'].shift(156)
df['p_Low_last_period156'] = df['p_l'].shift(156)
df['p_Close_last_period156'] = df['p_c'].shift(156)
#155 periods ago
df['p_Open_last_period155'] = df['p_o'].shift(155) 
df['p_High_last_period155'] = df['p_h'].shift(155)
df['p_Low_last_period155'] = df['p_l'].shift(155)
df['p_Close_last_period155'] = df['p_c'].shift(155)
#154 periods ago
df['p_Open_last_period154'] = df['p_o'].shift(154) 
df['p_High_last_period154'] = df['p_h'].shift(154)
df['p_Low_last_period154'] = df['p_l'].shift(154)
df['p_Close_last_period154'] = df['p_c'].shift(154)
#153 periods ago
df['p_Open_last_period153'] = df['p_o'].shift(153) 
df['p_High_last_period153'] = df['p_h'].shift(153)
df['p_Low_last_period153'] = df['p_l'].shift(153)
df['p_Close_last_period153'] = df['p_c'].shift(153)
#152 periods ago
df['p_Open_last_period152'] = df['p_o'].shift(152) 
df['p_High_last_period152'] = df['p_h'].shift(152)
df['p_Low_last_period152'] = df['p_l'].shift(152)
df['p_Close_last_period152'] = df['p_c'].shift(152)
#151 periods ago
df['p_Open_last_period151'] = df['p_o'].shift(151) 
df['p_High_last_period151'] = df['p_h'].shift(151)
df['p_Low_last_period151'] = df['p_l'].shift(151)
df['p_Close_last_period151'] = df['p_c'].shift(151)
#150 periods ago
df['p_Open_last_period150'] = df['p_o'].shift(150) 
df['p_High_last_period150'] = df['p_h'].shift(150)
df['p_Low_last_period150'] = df['p_l'].shift(150)
df['p_Close_last_period150'] = df['p_c'].shift(150)
#149 periods ago
df['p_Open_last_period149'] = df['p_o'].shift(149) 
df['p_High_last_period149'] = df['p_h'].shift(149)
df['p_Low_last_period149'] = df['p_l'].shift(149)
df['p_Close_last_period149'] = df['p_c'].shift(149)
#148 periods ago
df['p_Open_last_period148'] = df['p_o'].shift(148) 
df['p_High_last_period148'] = df['p_h'].shift(148)
df['p_Low_last_period148'] = df['p_l'].shift(148)
df['p_Close_last_period148'] = df['p_c'].shift(148)
#147 periods ago
df['p_Open_last_period147'] = df['p_o'].shift(147) 
df['p_High_last_period147'] = df['p_h'].shift(147)
df['p_Low_last_period147'] = df['p_l'].shift(147)
df['p_Close_last_period147'] = df['p_c'].shift(147)
#146 periods ago
df['p_Open_last_period146'] = df['p_o'].shift(146) 
df['p_High_last_period146'] = df['p_h'].shift(146)
df['p_Low_last_period146'] = df['p_l'].shift(146)
df['p_Close_last_period146'] = df['p_c'].shift(146)
#145 periods ago
df['p_Open_last_period145'] = df['p_o'].shift(145) 
df['p_High_last_period145'] = df['p_h'].shift(145)
df['p_Low_last_period145'] = df['p_l'].shift(145)
df['p_Close_last_period145'] = df['p_c'].shift(145)
#144 periods ago
df['p_Open_last_period144'] = df['p_o'].shift(144) 
df['p_High_last_period144'] = df['p_h'].shift(144)
df['p_Low_last_period144'] = df['p_l'].shift(144)
df['p_Close_last_period144'] = df['p_c'].shift(144)
#143 periods ago
df['p_Open_last_period143'] = df['p_o'].shift(143) 
df['p_High_last_period143'] = df['p_h'].shift(143)
df['p_Low_last_period143'] = df['p_l'].shift(143)
df['p_Close_last_period143'] = df['p_c'].shift(143)
#142 periods ago
df['p_Open_last_period142'] = df['p_o'].shift(142) 
df['p_High_last_period142'] = df['p_h'].shift(142)
df['p_Low_last_period142'] = df['p_l'].shift(142)
df['p_Close_last_period142'] = df['p_c'].shift(142)
#141 periods ago
df['p_Open_last_period141'] = df['p_o'].shift(141) 
df['p_High_last_period141'] = df['p_h'].shift(141)
df['p_Low_last_period141'] = df['p_l'].shift(141)
df['p_Close_last_period141'] = df['p_c'].shift(141)
#140 periods ago
df['p_Open_last_period140'] = df['p_o'].shift(140) 
df['p_High_last_period140'] = df['p_h'].shift(140)
df['p_Low_last_period140'] = df['p_l'].shift(140)
df['p_Close_last_period140'] = df['p_c'].shift(140)
#139 periods ago
df['p_Open_last_period139'] = df['p_o'].shift(139) 
df['p_High_last_period139'] = df['p_h'].shift(139)
df['p_Low_last_period139'] = df['p_l'].shift(139)
df['p_Close_last_period139'] = df['p_c'].shift(139)
#138 periods ago
df['p_Open_last_period138'] = df['p_o'].shift(138) 
df['p_High_last_period138'] = df['p_h'].shift(138)
df['p_Low_last_period138'] = df['p_l'].shift(138)
df['p_Close_last_period138'] = df['p_c'].shift(138)
#137 periods ago
df['p_Open_last_period137'] = df['p_o'].shift(137) 
df['p_High_last_period137'] = df['p_h'].shift(137)
df['p_Low_last_period137'] = df['p_l'].shift(137)
df['p_Close_last_period137'] = df['p_c'].shift(137)
#136 periods ago
df['p_Open_last_period136'] = df['p_o'].shift(136) 
df['p_High_last_period136'] = df['p_h'].shift(136)
df['p_Low_last_period136'] = df['p_l'].shift(136)
df['p_Close_last_period136'] = df['p_c'].shift(136)
#135 periods ago
df['p_Open_last_period135'] = df['p_o'].shift(135) 
df['p_High_last_period135'] = df['p_h'].shift(135)
df['p_Low_last_period135'] = df['p_l'].shift(135)
df['p_Close_last_period135'] = df['p_c'].shift(135)
#134 periods ago
df['p_Open_last_period134'] = df['p_o'].shift(134) 
df['p_High_last_period134'] = df['p_h'].shift(134)
df['p_Low_last_period134'] = df['p_l'].shift(134)
df['p_Close_last_period134'] = df['p_c'].shift(134)
#133 periods ago
df['p_Open_last_period133'] = df['p_o'].shift(133) 
df['p_High_last_period133'] = df['p_h'].shift(133)
df['p_Low_last_period133'] = df['p_l'].shift(133)
df['p_Close_last_period133'] = df['p_c'].shift(133)
#132 periods ago
df['p_Open_last_period132'] = df['p_o'].shift(132) 
df['p_High_last_period132'] = df['p_h'].shift(132)
df['p_Low_last_period132'] = df['p_l'].shift(132)
df['p_Close_last_period132'] = df['p_c'].shift(132)
#131 periods ago
df['p_Open_last_period131'] = df['p_o'].shift(131) 
df['p_High_last_period131'] = df['p_h'].shift(131)
df['p_Low_last_period131'] = df['p_l'].shift(131)
df['p_Close_last_period131'] = df['p_c'].shift(131)
#130 periods ago
df['p_Open_last_period130'] = df['p_o'].shift(130) 
df['p_High_last_period130'] = df['p_h'].shift(130)
df['p_Low_last_period130'] = df['p_l'].shift(130)
df['p_Close_last_period130'] = df['p_c'].shift(130)
#129 periods ago
df['p_Open_last_period129'] = df['p_o'].shift(129) 
df['p_High_last_period129'] = df['p_h'].shift(129)
df['p_Low_last_period129'] = df['p_l'].shift(129)
df['p_Close_last_period129'] = df['p_c'].shift(129)
#128 periods ago
df['p_Open_last_period128'] = df['p_o'].shift(128) 
df['p_High_last_period128'] = df['p_h'].shift(128)
df['p_Low_last_period128'] = df['p_l'].shift(128)
df['p_Close_last_period128'] = df['p_c'].shift(128)
#127 periods ago
df['p_Open_last_period127'] = df['p_o'].shift(127) 
df['p_High_last_period127'] = df['p_h'].shift(127)
df['p_Low_last_period127'] = df['p_l'].shift(127)
df['p_Close_last_period127'] = df['p_c'].shift(127)
#126 periods ago
df['p_Open_last_period126'] = df['p_o'].shift(126) 
df['p_High_last_period126'] = df['p_h'].shift(126)
df['p_Low_last_period126'] = df['p_l'].shift(126)
df['p_Close_last_period126'] = df['p_c'].shift(126)
#125 periods ago
df['p_Open_last_period125'] = df['p_o'].shift(125) 
df['p_High_last_period125'] = df['p_h'].shift(125)
df['p_Low_last_period125'] = df['p_l'].shift(125)
df['p_Close_last_period125'] = df['p_c'].shift(125)
#124 periods ago
df['p_Open_last_period124'] = df['p_o'].shift(124) 
df['p_High_last_period124'] = df['p_h'].shift(124)
df['p_Low_last_period124'] = df['p_l'].shift(124)
df['p_Close_last_period124'] = df['p_c'].shift(124)
#123 periods ago
df['p_Open_last_period123'] = df['p_o'].shift(123) 
df['p_High_last_period123'] = df['p_h'].shift(123)
df['p_Low_last_period123'] = df['p_l'].shift(123)
df['p_Close_last_period123'] = df['p_c'].shift(123)
#122 periods ago
df['p_Open_last_period122'] = df['p_o'].shift(122) 
df['p_High_last_period122'] = df['p_h'].shift(122)
df['p_Low_last_period122'] = df['p_l'].shift(122)
df['p_Close_last_period122'] = df['p_c'].shift(122)
#121 periods ago
df['p_Open_last_period121'] = df['p_o'].shift(121) 
df['p_High_last_period121'] = df['p_h'].shift(21)
df['p_Low_last_period121'] = df['p_l'].shift(121)
df['p_Close_last_period121'] = df['p_c'].shift(121)
#120 periods ago
df['p_Open_last_period120'] = df['p_o'].shift(120) 
df['p_High_last_period120'] = df['p_h'].shift(120)
df['p_Low_last_period120'] = df['p_l'].shift(120)
df['p_Close_last_period120'] = df['p_c'].shift(120)
#119 periods ago
df['p_Open_last_period119'] = df['p_o'].shift(119) 
df['p_High_last_period119'] = df['p_h'].shift(119)
df['p_Low_last_period119'] = df['p_l'].shift(119)
df['p_Close_last_period119'] = df['p_c'].shift(119)
#118 periods ago
df['p_Open_last_period118'] = df['p_o'].shift(118) 
df['p_High_last_period118'] = df['p_h'].shift(118)
df['p_Low_last_period118'] = df['p_l'].shift(118)
df['p_Close_last_period118'] = df['p_c'].shift(118)
#117 periods ago
df['p_Open_last_period117'] = df['p_o'].shift(117) 
df['p_High_last_period117'] = df['p_h'].shift(117)
df['p_Low_last_period117'] = df['p_l'].shift(117)
df['p_Close_last_period117'] = df['p_c'].shift(117)
#116 periods ago
df['p_Open_last_period116'] = df['p_o'].shift(116) 
df['p_High_last_period116'] = df['p_h'].shift(116)
df['p_Low_last_period116'] = df['p_l'].shift(116)
df['p_Close_last_period116'] = df['p_c'].shift(116)
#115 periods ago
df['p_Open_last_period115'] = df['p_o'].shift(115) 
df['p_High_last_period115'] = df['p_h'].shift(115)
df['p_Low_last_period115'] = df['p_l'].shift(115)
df['p_Close_last_period115'] = df['p_c'].shift(115)
#114 periods ago
df['p_Open_last_period114'] = df['p_o'].shift(114) 
df['p_High_last_period114'] = df['p_h'].shift(114)
df['p_Low_last_period1014'] = df['p_l'].shift(114)
df['p_Close_last_period114'] = df['p_c'].shift(114)
#113 periods ago
df['p_Open_last_period113'] = df['p_o'].shift(113) 
df['p_High_last_period113'] = df['p_h'].shift(113)
df['p_Low_last_period113'] = df['p_l'].shift(113)
df['p_Close_last_period113'] = df['p_c'].shift(113)
#112 periods ago
df['p_Open_last_period112'] = df['p_o'].shift(112) 
df['p_High_last_period112'] = df['p_h'].shift(112)
df['p_Low_last_period112'] = df['p_l'].shift(112)
df['p_Close_last_period112'] = df['p_c'].shift(112)
#111 periods ago
df['p_Open_last_period111'] = df['p_o'].shift(111) 
df['p_High_last_period111'] = df['p_h'].shift(111)
df['p_Low_last_period111'] = df['p_l'].shift(111)
df['p_Close_last_period111'] = df['p_c'].shift(111)
#110 periods ago
df['p_Open_last_period110'] = df['p_o'].shift(110) 
df['p_High_last_period110'] = df['p_h'].shift(110)
df['p_Low_last_period110'] = df['p_l'].shift(110)
df['p_Close_last_period110'] = df['p_c'].shift(110)
#109 periods ago
df['p_Open_last_period109'] = df['p_o'].shift(109) 
df['p_High_last_period109'] = df['p_h'].shift(109)
df['p_Low_last_period109'] = df['p_l'].shift(109)
df['p_Close_last_period109'] = df['p_c'].shift(109)
#108 periods ago
df['p_Open_last_period108'] = df['p_o'].shift(108) 
df['p_High_last_period108'] = df['p_h'].shift(108)
df['p_Low_last_period108'] = df['p_l'].shift(108)
df['p_Close_last_period108'] = df['p_c'].shift(108)
#107 periods ago
df['p_Open_last_period107'] = df['p_o'].shift(107) 
df['p_High_last_period107'] = df['p_h'].shift(107)
df['p_Low_last_period107'] = df['p_l'].shift(107)
df['p_Close_last_period107'] = df['p_c'].shift(107)
#106 periods ago
df['p_Open_last_period106'] = df['p_o'].shift(106) 
df['p_High_last_period106'] = df['p_h'].shift(106)
df['p_Low_last_period106'] = df['p_l'].shift(106)
df['p_Close_last_period106'] = df['p_c'].shift(106)
#105 periods ago
df['p_Open_last_period105'] = df['p_o'].shift(105) 
df['p_High_last_period105'] = df['p_h'].shift(105)
df['p_Low_last_period105'] = df['p_l'].shift(105)
df['p_Close_last_period105'] = df['p_c'].shift(105)
#104 periods ago
df['p_Open_last_period104'] = df['p_o'].shift(104) 
df['p_High_last_period104'] = df['p_h'].shift(104)
df['p_Low_last_period104'] = df['p_l'].shift(104)
df['p_Close_last_period104'] = df['p_c'].shift(104)
#103 periods ago
df['p_Open_last_period103'] = df['p_o'].shift(103) 
df['p_High_last_period103'] = df['p_h'].shift(103)
df['p_Low_last_period103'] = df['p_l'].shift(103)
df['p_Close_last_period103'] = df['p_c'].shift(103)
#102 periods ago
df['p_Open_last_period102'] = df['p_o'].shift(102) 
df['p_High_last_period102'] = df['p_h'].shift(102)
df['p_Low_last_period102'] = df['p_l'].shift(102)
df['p_Close_last_period102'] = df['p_c'].shift(102)
#101 periods ago
df['p_Open_last_period101'] = df['p_o'].shift(101) 
df['p_High_last_period101'] = df['p_h'].shift(101)
df['p_Low_last_period101'] = df['p_l'].shift(101)
df['p_Close_last_period101'] = df['p_c'].shift(101)
#100 periods ago
df['p_Open_last_period100'] = df['p_o'].shift(100) 
df['p_High_last_period100'] = df['p_h'].shift(100)
df['p_Low_last_period100'] = df['p_l'].shift(100)
df['p_Close_last_period100'] = df['p_c'].shift(100)
#99 periods ago
df['p_Open_last_period99'] = df['p_o'].shift(99) 
df['p_High_last_period99'] = df['p_h'].shift(99)
df['p_Low_last_period99'] = df['p_l'].shift(99)
df['p_Close_last_period99'] = df['p_c'].shift(99)
#98 periods ago
df['p_Open_last_period98'] = df['p_o'].shift(98) 
df['p_High_last_period98'] = df['p_h'].shift(98)
df['p_Low_last_period98'] = df['p_l'].shift(98)
df['p_Close_last_period98'] = df['p_c'].shift(98)
#97 periods ago
df['p_Open_last_period97'] = df['p_o'].shift(97) 
df['p_High_last_period97'] = df['p_h'].shift(97)
df['p_Low_last_period97'] = df['p_l'].shift(97)
df['p_Close_last_period97'] = df['p_c'].shift(97)
#96 periods ago
df['p_Open_last_period96'] = df['p_o'].shift(96) 
df['p_High_last_period96'] = df['p_h'].shift(96)
df['p_Low_last_period96'] = df['p_l'].shift(96)
df['p_Close_last_period96'] = df['p_c'].shift(96)
#95 periods ago
df['p_Open_last_period95'] = df['p_o'].shift(95) 
df['p_High_last_period95'] = df['p_h'].shift(95)
df['p_Low_last_period95'] = df['p_l'].shift(95)
df['p_Close_last_period95'] = df['p_c'].shift(95)
#94 periods ago
df['p_Open_last_period94'] = df['p_o'].shift(94) 
df['p_High_last_period94'] = df['p_h'].shift(94)
df['p_Low_last_period94'] = df['p_l'].shift(94)
df['p_Close_last_period94'] = df['p_c'].shift(94)
#93 periods ago
df['p_Open_last_period93'] = df['p_o'].shift(93) 
df['p_High_last_period93'] = df['p_h'].shift(93)
df['p_Low_last_period93'] = df['p_l'].shift(93)
df['p_Close_last_period93'] = df['p_c'].shift(93)
#92 periods ago
df['p_Open_last_period92'] = df['p_o'].shift(92) 
df['p_High_last_period92'] = df['p_h'].shift(92)
df['p_Low_last_period92'] = df['p_l'].shift(92)
df['p_Close_last_period92'] = df['p_c'].shift(92)
#91 periods ago
df['p_Open_last_period91'] = df['p_o'].shift(91) 
df['p_High_last_period91'] = df['p_h'].shift(91)
df['p_Low_last_period91'] = df['p_l'].shift(91)
df['p_Close_last_period91'] = df['p_c'].shift(91)
#90 periods ago
df['p_Open_last_period90'] = df['p_o'].shift(90) 
df['p_High_last_period90'] = df['p_h'].shift(90)
df['p_Low_last_period90'] = df['p_l'].shift(90)
df['p_Close_last_period90'] = df['p_c'].shift(90)
#89 periods ago
df['p_Open_last_period89'] = df['p_o'].shift(89) 
df['p_High_last_period89'] = df['p_h'].shift(89)
df['p_Low_last_period89'] = df['p_l'].shift(89)
df['p_Close_last_period89'] = df['p_c'].shift(89)
#88 periods ago
df['p_Open_last_period88'] = df['p_o'].shift(88) 
df['p_High_last_period88'] = df['p_h'].shift(88)
df['p_Low_last_period88'] = df['p_l'].shift(88)
df['p_Close_last_period88'] = df['p_c'].shift(88)
#87 periods ago
df['p_Open_last_period87'] = df['p_o'].shift(87) 
df['p_High_last_period87'] = df['p_h'].shift(87)
df['p_Low_last_period87'] = df['p_l'].shift(87)
df['p_Close_last_period87'] = df['p_c'].shift(87)
#86 periods ago
df['p_Open_last_period86'] = df['p_o'].shift(86) 
df['p_High_last_period86'] = df['p_h'].shift(86)
df['p_Low_last_period86'] = df['p_l'].shift(86)
df['p_Close_last_period86'] = df['p_c'].shift(86)
#85 periods ago
df['p_Open_last_period85'] = df['p_o'].shift(85) 
df['p_High_last_period85'] = df['p_h'].shift(85)
df['p_Low_last_period85'] = df['p_l'].shift(85)
df['p_Close_last_period85'] = df['p_c'].shift(85)
#84 periods ago
df['p_Open_last_period84'] = df['p_o'].shift(84) 
df['p_High_last_period84'] = df['p_h'].shift(84)
df['p_Low_last_period84'] = df['p_l'].shift(84)
df['p_Close_last_period84'] = df['p_c'].shift(84)
#83 periods ago
df['p_Open_last_period83'] = df['p_o'].shift(83) 
df['p_High_last_period83'] = df['p_h'].shift(83)
df['p_Low_last_period83'] = df['p_l'].shift(83)
df['p_Close_last_period83'] = df['p_c'].shift(83)
#82 periods ago
df['p_Open_last_period82'] = df['p_o'].shift(82) 
df['p_High_last_period82'] = df['p_h'].shift(82)
df['p_Low_last_period82'] = df['p_l'].shift(82)
df['p_Close_last_period82'] = df['p_c'].shift(82)
#81 periods ago
df['p_Open_last_period81'] = df['p_o'].shift(81) 
df['p_High_last_period81'] = df['p_h'].shift(81)
df['p_Low_last_period81'] = df['p_l'].shift(81)
df['p_Close_last_period81'] = df['p_c'].shift(81)
#80 periods ago
df['p_Open_last_period80'] = df['p_o'].shift(80) 
df['p_High_last_period80'] = df['p_h'].shift(80)
df['p_Low_last_period80'] = df['p_l'].shift(80)
df['p_Close_last_period80'] = df['p_c'].shift(80)
#79 periods ago
df['p_Open_last_period79'] = df['p_o'].shift(79) 
df['p_High_last_period79'] = df['p_h'].shift(79)
df['p_Low_last_period79'] = df['p_l'].shift(79)
df['p_Close_last_period79'] = df['p_c'].shift(79)
#78 periods ago
df['p_Open_last_period78'] = df['p_o'].shift(78) 
df['p_High_last_period78'] = df['p_h'].shift(78)
df['p_Low_last_period78'] = df['p_l'].shift(78)
df['p_Close_last_period78'] = df['p_c'].shift(78)
#77 periods ago
df['p_Open_last_period77'] = df['p_o'].shift(77) 
df['p_High_last_period77'] = df['p_h'].shift(77)
df['p_Low_last_period77'] = df['p_l'].shift(77)
df['p_Close_last_period77'] = df['p_c'].shift(77)
#76 periods ago
df['p_Open_last_period76'] = df['p_o'].shift(76) 
df['p_High_last_period76'] = df['p_h'].shift(76)
df['p_Low_last_period76'] = df['p_l'].shift(76)
df['p_Close_last_period76'] = df['p_c'].shift(76)
#75 periods ago
df['p_Open_last_period75'] = df['p_o'].shift(75) 
df['p_High_last_period75'] = df['p_h'].shift(75)
df['p_Low_last_period75'] = df['p_l'].shift(75)
df['p_Close_last_period75'] = df['p_c'].shift(75)
#74 periods ago
df['p_Open_last_period74'] = df['p_o'].shift(74) 
df['p_High_last_period74'] = df['p_h'].shift(74)
df['p_Low_last_period74'] = df['p_l'].shift(74)
df['p_Close_last_period74'] = df['p_c'].shift(74)
#73 periods ago
df['p_Open_last_period73'] = df['p_o'].shift(73) 
df['p_High_last_period73'] = df['p_h'].shift(73)
df['p_Low_last_period73'] = df['p_l'].shift(73)
df['p_Close_last_period73'] = df['p_c'].shift(73)
#72 periods ago
df['p_Open_last_period72'] = df['p_o'].shift(72) 
df['p_High_last_period72'] = df['p_h'].shift(72)
df['p_Low_last_period72'] = df['p_l'].shift(72)
df['p_Close_last_period72'] = df['p_c'].shift(72)
#71 periods ago
df['p_Open_last_period71'] = df['p_o'].shift(71) 
df['p_High_last_period71'] = df['p_h'].shift(71)
df['p_Low_last_period71'] = df['p_l'].shift(71)
df['p_Close_last_period71'] = df['p_c'].shift(71)
#70 periods ago
df['p_Open_last_period70'] = df['p_o'].shift(70) 
df['p_High_last_period70'] = df['p_h'].shift(70)
df['p_Low_last_period70'] = df['p_l'].shift(70)
df['p_Close_last_period70'] = df['p_c'].shift(70)
#69 periods ago
df['p_Open_last_period69'] = df['p_o'].shift(69) 
df['p_High_last_period69'] = df['p_h'].shift(69)
df['p_Low_last_period69'] = df['p_l'].shift(69)
df['p_Close_last_period69'] = df['p_c'].shift(69)
#68 periods ago
df['p_Open_last_period68'] = df['p_o'].shift(68) 
df['p_High_last_period68'] = df['p_h'].shift(68)
df['p_Low_last_period68'] = df['p_l'].shift(68)
df['p_Close_last_period68'] = df['p_c'].shift(68)
#67 periods ago
df['p_Open_last_period67'] = df['p_o'].shift(67) 
df['p_High_last_period67'] = df['p_h'].shift(67)
df['p_Low_last_period67'] = df['p_l'].shift(67)
df['p_Close_last_period67'] = df['p_c'].shift(67)
#66 periods ago
df['p_Open_last_period66'] = df['p_o'].shift(66) 
df['p_High_last_period66'] = df['p_h'].shift(66)
df['p_Low_last_period66'] = df['p_l'].shift(66)
df['p_Close_last_period66'] = df['p_c'].shift(66)
#65 periods ago
df['p_Open_last_period65'] = df['p_o'].shift(65) 
df['p_High_last_period65'] = df['p_h'].shift(65)
df['p_Low_last_period65'] = df['p_l'].shift(65)
df['p_Close_last_period65'] = df['p_c'].shift(65)
#64 periods ago
df['p_Open_last_period64'] = df['p_o'].shift(64) 
df['p_High_last_period64'] = df['p_h'].shift(64)
df['p_Low_last_period64'] = df['p_l'].shift(64)
df['p_Close_last_period64'] = df['p_c'].shift(64)
#63 periods ago
df['p_Open_last_period63'] = df['p_o'].shift(63) 
df['p_High_last_period63'] = df['p_h'].shift(63)
df['p_Low_last_period63'] = df['p_l'].shift(63)
df['p_Close_last_period63'] = df['p_c'].shift(63)
#62 periods ago
df['p_Open_last_period62'] = df['p_o'].shift(62) 
df['p_High_last_period62'] = df['p_h'].shift(62)
df['p_Low_last_period62'] = df['p_l'].shift(62)
df['p_Close_last_period62'] = df['p_c'].shift(62)
#61 periods ago
df['p_Open_last_period61'] = df['p_o'].shift(61) 
df['p_High_last_period61'] = df['p_h'].shift(61)
df['p_Low_last_period61'] = df['p_l'].shift(61)
df['p_Close_last_period61'] = df['p_c'].shift(61)
#60 periods ago
df['p_Open_last_period60'] = df['p_o'].shift(60) 
df['p_High_last_period60'] = df['p_h'].shift(60)
df['p_Low_last_period60'] = df['p_l'].shift(60)
df['p_Close_last_period60'] = df['p_c'].shift(60)
#59 periods ago
df['p_Open_last_period59'] = df['p_o'].shift(59) 
df['p_High_last_period59'] = df['p_h'].shift(59)
df['p_Low_last_period59'] = df['p_l'].shift(59)
df['p_Close_last_period59'] = df['p_c'].shift(59)
#58 periods ago
df['p_Open_last_period58'] = df['p_o'].shift(58) 
df['p_High_last_period58'] = df['p_h'].shift(58)
df['p_Low_last_period58'] = df['p_l'].shift(58)
df['p_Close_last_period58'] = df['p_c'].shift(58)
#57 periods ago
df['p_Open_last_period57'] = df['p_o'].shift(57) 
df['p_High_last_period57'] = df['p_h'].shift(57)
df['p_Low_last_period57'] = df['p_l'].shift(57)
df['p_Close_last_period57'] = df['p_c'].shift(57)
#56 periods ago
df['p_Open_last_period56'] = df['p_o'].shift(56) 
df['p_High_last_period56'] = df['p_h'].shift(56)
df['p_Low_last_period56'] = df['p_l'].shift(56)
df['p_Close_last_period56'] = df['p_c'].shift(56)
#55 periods ago
df['p_Open_last_period55'] = df['p_o'].shift(55) 
df['p_High_last_period55'] = df['p_h'].shift(55)
df['p_Low_last_period55'] = df['p_l'].shift(55)
df['p_Close_last_period55'] = df['p_c'].shift(55)
#54 periods ago
df['p_Open_last_period54'] = df['p_o'].shift(54) 
df['p_High_last_period54'] = df['p_h'].shift(54)
df['p_Low_last_period54'] = df['p_l'].shift(54)
df['p_Close_last_period54'] = df['p_c'].shift(54)
#53 periods ago
df['p_Open_last_period53'] = df['p_o'].shift(53) 
df['p_High_last_period53'] = df['p_h'].shift(53)
df['p_Low_last_period53'] = df['p_l'].shift(53)
df['p_Close_last_period53'] = df['p_c'].shift(53)
#52 periods ago
df['p_Open_last_period52'] = df['p_o'].shift(52) 
df['p_High_last_period52'] = df['p_h'].shift(52)
df['p_Low_last_period52'] = df['p_l'].shift(52)
df['p_Close_last_period52'] = df['p_c'].shift(52)
#51 periods ago
df['p_Open_last_period51'] = df['p_o'].shift(51) 
df['p_High_last_period51'] = df['p_h'].shift(51)
df['p_Low_last_period51'] = df['p_l'].shift(51)
df['p_Close_last_period51'] = df['p_c'].shift(51)
#50 periods ago
df['p_Open_last_period50'] = df['p_o'].shift(50) 
df['p_High_last_period50'] = df['p_h'].shift(50)
df['p_Low_last_period50'] = df['p_l'].shift(50)
df['p_Close_last_period50'] = df['p_c'].shift(50)
#49 periods ago
df['p_Open_last_period49'] = df['p_o'].shift(49) 
df['p_High_last_period49'] = df['p_h'].shift(49)
df['p_Low_last_period49'] = df['p_l'].shift(49)
df['p_Close_last_period49'] = df['p_c'].shift(49)
#48 periods ago
df['p_Open_last_period48'] = df['p_o'].shift(48) 
df['p_High_last_period48'] = df['p_h'].shift(48)
df['p_Low_last_period48'] = df['p_l'].shift(48)
df['p_Close_last_period48'] = df['p_c'].shift(48)
#47 periods ago
df['p_Open_last_period47'] = df['p_o'].shift(47) 
df['p_High_last_period47'] = df['p_h'].shift(47)
df['p_Low_last_period47'] = df['p_l'].shift(47)
df['p_Close_last_period47'] = df['p_c'].shift(47)
#46 periods ago
df['p_Open_last_period46'] = df['p_o'].shift(46) 
df['p_High_last_period46'] = df['p_h'].shift(46)
df['p_Low_last_period46'] = df['p_l'].shift(46)
df['p_Close_last_period46'] = df['p_c'].shift(46)
#45 periods ago
df['p_Open_last_period45'] = df['p_o'].shift(45) 
df['p_High_last_period45'] = df['p_h'].shift(45)
df['p_Low_last_period45'] = df['p_l'].shift(45)
df['p_Close_last_period45'] = df['p_c'].shift(45)
#44 periods ago
df['p_Open_last_period44'] = df['p_o'].shift(44) 
df['p_High_last_period44'] = df['p_h'].shift(44)
df['p_Low_last_period44'] = df['p_l'].shift(44)
df['p_Close_last_period44'] = df['p_c'].shift(44)
#43 periods ago
df['p_Open_last_period43'] = df['p_o'].shift(43) 
df['p_High_last_period43'] = df['p_h'].shift(43)
df['p_Low_last_period43'] = df['p_l'].shift(43)
df['p_Close_last_period43'] = df['p_c'].shift(43)
#42 periods ago
df['p_Open_last_period42'] = df['p_o'].shift(42) 
df['p_High_last_period42'] = df['p_h'].shift(42)
df['p_Low_last_period42'] = df['p_l'].shift(42)
df['p_Close_last_period42'] = df['p_c'].shift(42)
#41 periods ago
df['p_Open_last_period41'] = df['p_o'].shift(41) 
df['p_High_last_period41'] = df['p_h'].shift(41)
df['p_Low_last_period41'] = df['p_l'].shift(41)
df['p_Close_last_period41'] = df['p_c'].shift(41)
#40 periods ago
df['p_Open_last_period40'] = df['p_o'].shift(40) 
df['p_High_last_period40'] = df['p_h'].shift(40)
df['p_Low_last_period40'] = df['p_l'].shift(40)
df['p_Close_last_period40'] = df['p_c'].shift(40)
#39 periods ago
df['p_Open_last_period39'] = df['p_o'].shift(39) 
df['p_High_last_period39'] = df['p_h'].shift(39)
df['p_Low_last_period39'] = df['p_l'].shift(39)
df['p_Close_last_period39'] = df['p_c'].shift(39)
#38 periods ago
df['p_Open_last_period38'] = df['p_o'].shift(38) 
df['p_High_last_period38'] = df['p_h'].shift(38)
df['p_Low_last_period38'] = df['p_l'].shift(38)
df['p_Close_last_period38'] = df['p_c'].shift(38)
#37 periods ago
df['p_Open_last_period37'] = df['p_o'].shift(37) 
df['p_High_last_period37'] = df['p_h'].shift(37)
df['p_Low_last_period37'] = df['p_l'].shift(37)
df['p_Close_last_period37'] = df['p_c'].shift(37)
#36 periods ago
df['p_Open_last_period36'] = df['p_o'].shift(36) 
df['p_High_last_period36'] = df['p_h'].shift(36)
df['p_Low_last_period36'] = df['p_l'].shift(36)
df['p_Close_last_period36'] = df['p_c'].shift(36)
#35 periods ago
df['p_Open_last_period35'] = df['p_o'].shift(35) 
df['p_High_last_period35'] = df['p_h'].shift(35)
df['p_Low_last_period35'] = df['p_l'].shift(35)
df['p_Close_last_period35'] = df['p_c'].shift(35)
#34 periods ago
df['p_Open_last_period34'] = df['p_o'].shift(34) 
df['p_High_last_period34'] = df['p_h'].shift(34)
df['p_Low_last_period34'] = df['p_l'].shift(34)
df['p_Close_last_period34'] = df['p_c'].shift(34)
#33 periods ago
df['p_Open_last_period33'] = df['p_o'].shift(33) 
df['p_High_last_period33'] = df['p_h'].shift(33)
df['p_Low_last_period33'] = df['p_l'].shift(33)
df['p_Close_last_period33'] = df['p_c'].shift(33)
#32 periods ago
df['p_Open_last_period32'] = df['p_o'].shift(32) 
df['p_High_last_period32'] = df['p_h'].shift(32)
df['p_Low_last_period32'] = df['p_l'].shift(32)
df['p_Close_last_period32'] = df['p_c'].shift(32)
#31 periods ago
df['p_Open_last_period31'] = df['p_o'].shift(31) 
df['p_High_last_period31'] = df['p_h'].shift(31)
df['p_Low_last_period31'] = df['p_l'].shift(31)
df['p_Close_last_period31'] = df['p_c'].shift(31)
#30 periods ago
df['p_Open_last_period30'] = df['p_o'].shift(30) 
df['p_High_last_period30'] = df['p_h'].shift(30)
df['p_Low_last_period30'] = df['p_l'].shift(30)
df['p_Close_last_period30'] = df['p_c'].shift(30) 
#29 periods ago 
df['p_Open_last_period29'] = df['p_o'].shift(29) 
df['p_High_last_period29'] = df['p_h'].shift(29)
df['p_Low_last_period29'] = df['p_l'].shift(29)
df['p_Close_last_period29'] = df['p_c'].shift(29)
#28 periods ago
df['p_Open_last_period28'] = df['p_o'].shift(28) 
df['p_High_last_period28'] = df['p_h'].shift(28)
df['p_Low_last_period28'] = df['p_l'].shift(28)
df['p_Close_last_period28'] = df['p_c'].shift(28) 
#27 periods ago
df['p_Open_last_period27'] = df['p_o'].shift(27) 
df['p_High_last_period27'] = df['p_h'].shift(27)
df['p_Low_last_period27'] = df['p_l'].shift(27)
df['p_Close_last_period27'] = df['p_c'].shift(27)
#26 periods ago
df['p_Open_last_period26'] = df['p_o'].shift(26) 
df['p_High_last_period26'] = df['p_h'].shift(26)
df['p_Low_last_period26'] = df['p_l'].shift(26)
df['p_Close_last_period26'] = df['p_c'].shift(26)
#25 periods ago
df['p_Open_last_period25'] = df['p_o'].shift(25) 
df['p_High_last_period25'] = df['p_h'].shift(25)
df['p_Low_last_period25'] = df['p_l'].shift(25)
df['p_Close_last_period25'] = df['p_c'].shift(25) 
#24 periods ago 
df['p_Open_last_period24'] = df['p_o'].shift(24) 
df['p_High_last_period24'] = df['p_h'].shift(24)
df['p_Low_last_period24'] = df['p_l'].shift(24)
df['p_Close_last_period24'] = df['p_c'].shift(24) 
#23 periods ago
df['p_Open_last_period23'] = df['p_o'].shift(23) 
df['p_High_last_period23'] = df['p_h'].shift(23)
df['p_Low_last_period23'] = df['p_l'].shift(23)
df['p_Close_last_period23'] = df['p_c'].shift(23) 
#22 periods ago
df['p_Open_last_period22'] = df['p_o'].shift(22) 
df['p_High_last_period22'] = df['p_h'].shift(22)
df['p_Low_last_period22'] = df['p_l'].shift(22)
df['p_Close_last_period22'] = df['p_c'].shift(22) 
#21 periods ago
df['p_Open_last_period21'] = df['p_o'].shift(21) 
df['p_High_last_period21'] = df['p_h'].shift(21)
df['p_Low_last_period21'] = df['p_l'].shift(21)
df['p_Close_last_period21'] = df['p_c'].shift(21)
#20 periods ago
df['p_Open_last_period20'] = df['p_o'].shift(20) 
df['p_High_last_period20'] = df['p_h'].shift(20)
df['p_Low_last_period20'] = df['p_l'].shift(20)
df['p_Close_last_period20'] = df['p_c'].shift(20) 
#19 periods ago 
df['p_Open_last_period19'] = df['p_o'].shift(19) 
df['p_High_last_period19'] = df['p_h'].shift(19)
df['p_Low_last_period19'] = df['p_l'].shift(19)
df['p_Close_last_period19'] = df['p_c'].shift(19)
#18 periods ago
df['p_Open_last_period18'] = df['p_o'].shift(18) 
df['p_High_last_period18'] = df['p_h'].shift(18)
df['p_Low_last_period18'] = df['p_l'].shift(18)
df['p_Close_last_period18'] = df['p_c'].shift(18) 
#17 periods ago
df['p_Open_last_period17'] = df['p_o'].shift(17) 
df['p_High_last_period17'] = df['p_h'].shift(17)
df['p_Low_last_period17'] = df['p_l'].shift(17)
df['p_Close_last_period17'] = df['p_c'].shift(17) 
#16 periods ago
df['p_Open_last_period16'] = df['p_o'].shift(16) 
df['p_High_last_period16'] = df['p_h'].shift(16)
df['p_Low_last_period16'] = df['p_l'].shift(16)
df['p_Close_last_period16'] = df['p_c'].shift(16)
#15 periods ago
df['p_Open_last_period15'] = df['p_o'].shift(15) 
df['p_High_last_period15'] = df['p_h'].shift(15)
df['p_Low_last_period15'] = df['p_l'].shift(15)
df['p_Close_last_period15'] = df['p_c'].shift(15)
#14 periods ago 
df['p_Open_last_period14'] = df['p_o'].shift(14) 
df['p_High_last_period14'] = df['p_h'].shift(14)
df['p_Low_last_period14'] = df['p_l'].shift(14)
df['p_Close_last_period14'] = df['p_c'].shift(14) 
#13 periods ago
df['p_Open_last_period13'] = df['p_o'].shift(13) 
df['p_High_last_period13'] = df['p_h'].shift(13)
df['p_Low_last_period13'] = df['p_l'].shift(13)
df['p_Close_last_period13'] = df['p_c'].shift(13) 
#12 periods ago
df['p_Open_last_period12'] = df['p_o'].shift(12) 
df['p_High_last_period12'] = df['p_h'].shift(12)
df['p_Low_last_period12'] = df['p_l'].shift(12)
df['p_Close_last_period12'] = df['p_c'].shift(12) 
#11 periods ago 
df['p_Open_last_period11'] = df['p_o'].shift(11) 
df['p_High_last_period11'] = df['p_h'].shift(11)
df['p_Low_last_period11'] = df['p_l'].shift(11)
df['p_Close_last_period11'] = df['p_c'].shift(11) 
#10 periods ago
df['p_Open_last_period10'] = df['p_o'].shift(10) 
df['p_High_last_period10'] = df['p_h'].shift(10)
df['p_Low_last_period10'] = df['p_l'].shift(10)
df['p_Close_last_period10'] = df['p_c'].shift(10)
#9 periods ago 
df['p_Open_last_period9'] = df['p_o'].shift(9) 
df['p_High_last_period9'] = df['p_h'].shift(9)
df['p_Low_last_period9'] = df['p_l'].shift(9)
df['p_Close_last_period9'] = df['p_c'].shift(9) 
#8 periods ago
df['p_Open_last_period8'] = df['p_o'].shift(8) 
df['p_High_last_period8'] = df['p_h'].shift(8)
df['p_Low_last_period8'] = df['p_l'].shift(8)
df['p_Close_last_period8'] = df['p_c'].shift(8)
#7 periods ago
df['p_Open_last_period7'] = df['p_o'].shift(7) 
df['p_High_last_period7'] = df['p_h'].shift(7)
df['p_Low_last_period7'] = df['p_l'].shift(7)
df['p_Close_last_period7'] = df['p_c'].shift(7) 
#6 periods ago 
df['p_Open_last_period6'] = df['p_o'].shift(6) 
df['p_High_last_period6'] = df['p_h'].shift(6)
df['p_Low_last_period6'] = df['p_l'].shift(6)
df['p_Close_last_period6'] = df['p_c'].shift(6)
#5 periods ago
df['p_Open_last_period5'] = df['p_o'].shift(5) 
df['p_High_last_period5'] = df['p_h'].shift(5)
df['p_Low_last_period5'] = df['p_l'].shift(5)
df['p_Close_last_period5'] = df['p_c'].shift(5) 
#4 periods ago 
df['p_Open_last_period4'] = df['p_o'].shift(4) 
df['p_High_last_period4'] = df['p_h'].shift(4)
df['p_Low_last_period4'] = df['p_l'].shift(4)
df['p_Close_last_period4'] = df['p_c'].shift(4) 
#3 periods ago
df['p_Open_last_period3'] = df['p_o'].shift(3) 
df['p_High_last_period3'] = df['p_h'].shift(3)
df['p_Low_last_period3'] = df['p_l'].shift(3)
df['p_Close_last_period3'] = df['p_c'].shift(3)
#2 periods ago
df['p_Open_last_period2'] = df['p_o'].shift(2) 
df['p_High_last_period2'] = df['p_h'].shift(2)
df['p_Low_last_period2'] = df['p_l'].shift(2)
df['p_Close_last_period2'] = df['p_c'].shift(2) 
#1 periods ago 
df['p_Open_last_period1'] = df['p_o'].shift(1)     
df['p_High_last_period1'] = df['p_h'].shift(1)
df['p_Low_last_period1'] = df['p_l'].shift(1)
df['p_Close_last_period1'] = df['p_c'].shift(1)       

#yesterday
df['p_Open_yesterday'] = df['p_o'].shift(0)     
df['p_High_yesterday'] = df['p_h'].shift(0)
df['p_Low_yesterday'] = df['p_l'].shift(0)
df['p_Close_yesterday'] = df['p_c'].shift(0) 

#Target
#1 periods forward 
df['p_High_next_period1'] = df['p_h'].shift(-1)
df['p_Low_next_period1'] = df['p_l'].shift(-1)
df['p_Close_next_period1'] = df['p_c'].shift(-1) 

#drop null and useless columns
df = df.dropna()
df = df.drop(['p_o','p_h','p_l','p_c'], axis=1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:132: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:135: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame

In [17]:
df

,p_Open_last_period230,p_High_last_period230,p_Low_last_period230,p_Close_last_period230,p_Open_last_period229,p_High_last_period229,p_Low_last_period229,p_Close_last_period229,p_Open_last_period228,p_High_last_period228,...,p_High_last_period1,p_Low_last_period1,p_Close_last_period1,p_Open_yesterday,p_High_yesterday,p_Low_yesterday,p_Close_yesterday,p_High_next_period1,p_Low_next_period1,p_Close_next_period1
230,357.730,357.730,357.730,357.730,357.810,357.810,357.810,357.810,357.860,357.860,...,320.360,320.360,320.360,314.980,314.980,314.980,314.980,314.960,314.960,314.960
231,357.810,357.810,357.810,357.810,357.860,357.860,357.860,357.860,357.870,357.870,...,314.980,314.980,314.980,314.960,314.960,314.960,314.960,315.010,315.010,315.010
232,357.860,357.860,357.860,357.860,357.870,357.870,357.870,357.870,357.820,357.820,...,314.960,314.960,314.960,315.010,315.010,315.010,315.010,315.210,315.210,315.210
233,357.870,357.870,357.870,357.870,357.820,357.820,357.820,357.820,357.950,357.950,...,315.010,315.010,315.010,315.210,315.210,315.210,315.210,314.960,314.960,314.960
234,357.820,357.820,357.820,357.820,357.950,357.950,357.950,357.950,358.060,358.060,...,315.210,315.210,315.210,314.960,314.960,314.960,314.960,315.010,315.010,315.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13231,110.220,110.332,109.941,110.211,110.095,110.352,109.716,110.314,110.313,110.791,...,130.558,128.753,130.184,130.184,130.808,130.093,130.553,131.348,130.112,130.319
13232,110.095,110.352,109.716,110.314,110.313,110.791,110.211,110.658,110.657,111.102,...,130.808,130.093,130.553,130.491,131.348,130.112,130.319,130.570,129.797,130.414
13233,110.313,110.791,110.211,110.658,110.657,111.102,110.567,110.949,110.948,111.115,...,131.348,130.112,130.319,130.316,130.570,129.797,130.414,130.811,129.443,129.967
13234,110.657,111.102,110.567,110.949,110.948,111.115,110.687,110.874,110.872,110.984,...,130.570,129.797,130.414,130.416,130.811,129.443,129.967,130.049,127.520,128.303


In [18]:
df_pre = pd.DataFrame(df[['p_Open_last_period1','p_High_last_period1','p_Low_last_period1','p_Close_last_period1','p_Close_next_period1']])

In [19]:
df_pre

,p_Open_last_period1,p_High_last_period1,p_Low_last_period1,p_Close_last_period1,p_Close_next_period1
230,320.360,320.360,320.360,320.360,314.960
231,314.980,314.980,314.980,314.980,315.010
232,314.960,314.960,314.960,314.960,315.210
233,315.010,315.010,315.010,315.010,314.960
234,315.210,315.210,315.210,315.210,315.010
...,...,...,...,...,...
13231,129.055,130.558,128.753,130.184,130.319
13232,130.184,130.808,130.093,130.553,130.414
13233,130.491,131.348,130.112,130.319,129.967
13234,130.316,130.570,129.797,130.414,128.303


In [20]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13006 entries, 230 to 13235
Data columns (total 927 columns):
 #    Column                  Dtype  
---   ------                  -----  
 0    p_Open_last_period230   float64
 1    p_High_last_period230   float64
 2    p_Low_last_period230    float64
 3    p_Close_last_period230  float64
 4    p_Open_last_period229   float64
 5    p_High_last_period229   float64
 6    p_Low_last_period229    float64
 7    p_Close_last_period229  float64
 8    p_Open_last_period228   float64
 9    p_High_last_period228   float64
 10   p_Low_last_period228    float64
 11   p_Close_last_period228  float64
 12   p_Open_last_period227   float64
 13   p_High_last_period227   float64
 14   p_Low_last_period227    float64
 15   p_Close_last_period227  float64
 16   p_Open_last_period226   float64
 17   p_High_last_period226   float64
 18   p_Low_last_period226    float64
 19   p_Close_last_period226  float64
 20   p_Open_last_period225   float64
 21   p_High_l

In [21]:
#create initial train and test data as pd.DF for close
#final test DF is last_row
df_train_close = pd.DataFrame(data=df.iloc[:-40,:])
df_test_close = pd.DataFrame(data=df.iloc[-40:,:])

#df_train_close = pd.DataFrame(data=df.iloc[:-40,[2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92]])
#df_test_close = pd.DataFrame(data=df.iloc[-40:,[2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59,62,65,68,71,74,77,80,83,86,89,92]])
print(df_train_close )
print(df_test_close )

       p_Open_last_period230  p_High_last_period230  p_Low_last_period230  \
230                  357.730                357.730               357.730   
231                  357.810                357.810               357.810   
232                  357.860                357.860               357.860   
233                  357.870                357.870               357.870   
234                  357.820                357.820               357.820   
...                      ...                    ...                   ...   
13191                107.962                108.145               107.477   
13192                107.887                108.197               107.643   
13193                108.083                108.777               108.055   
13194                108.667                109.075               108.571   
13195                108.596                109.223               108.436   

       p_Close_last_period230  p_Open_last_period229  p_High_last_period229

In [22]:
print(df_train_close.isna().sum())
print(df_test_close.isna().sum())

p_Open_last_period230     0
p_High_last_period230     0
p_Low_last_period230      0
p_Close_last_period230    0
p_Open_last_period229     0
                         ..
p_Low_yesterday           0
p_Close_yesterday         0
p_High_next_period1       0
p_Low_next_period1        0
p_Close_next_period1      0
Length: 927, dtype: int64
p_Open_last_period230     0
p_High_last_period230     0
p_Low_last_period230      0
p_Close_last_period230    0
p_Open_last_period229     0
                         ..
p_Low_yesterday           0
p_Close_yesterday         0
p_High_next_period1       0
p_Low_next_period1        0
p_Close_next_period1      0
Length: 927, dtype: int64


In [23]:
print(df_train_close.info())
print(df_test_close.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12966 entries, 230 to 13195
Columns: 927 entries, p_Open_last_period230 to p_Close_next_period1
dtypes: float64(927)
memory usage: 91.8 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 13196 to 13235
Columns: 927 entries, p_Open_last_period230 to p_Close_next_period1
dtypes: float64(927)
memory usage: 290.0 KB
None


In [24]:
train_close = df_train_close
train_close=train_close
train_close.tail(15)

,p_Open_last_period230,p_High_last_period230,p_Low_last_period230,p_Close_last_period230,p_Open_last_period229,p_High_last_period229,p_Low_last_period229,p_Close_last_period229,p_Open_last_period228,p_High_last_period228,...,p_High_last_period1,p_Low_last_period1,p_Close_last_period1,p_Open_yesterday,p_High_yesterday,p_Low_yesterday,p_Close_yesterday,p_High_next_period1,p_Low_next_period1,p_Close_next_period1
13181,109.268,109.960,109.203,109.658,109.605,109.769,109.245,109.378,109.384,109.752,...,115.697,114.407,115.530,115.500,115.760,115.149,115.526,115.782,114.858,114.969
13182,109.605,109.769,109.245,109.378,109.384,109.752,109.018,109.059,109.054,109.095,...,115.760,115.149,115.526,115.225,115.782,114.858,114.969,115.288,114.698,114.911
13183,109.384,109.752,109.018,109.059,109.054,109.095,108.751,108.916,108.912,108.965,...,115.782,114.858,114.969,114.944,115.288,114.698,114.911,115.686,114.788,115.514
13184,109.054,109.095,108.751,108.916,108.912,108.965,108.611,108.762,108.762,108.964,...,115.288,114.698,114.911,114.891,115.686,114.788,115.514,115.807,115.383,115.454
13185,108.912,108.965,108.611,108.762,108.762,108.964,108.610,108.793,108.726,108.833,...,115.686,114.788,115.514,115.483,115.807,115.383,115.454,115.555,114.649,114.854
13186,108.762,108.964,108.610,108.793,108.726,108.833,108.009,108.153,108.138,108.546,...,115.807,115.383,115.454,115.446,115.555,114.649,114.854,115.474,114.813,115.310
13187,108.726,108.833,108.009,108.153,108.138,108.546,107.970,108.101,108.099,108.281,...,115.555,114.649,114.854,114.897,115.474,114.813,115.310,115.791,115.229,115.662
13188,108.138,108.546,107.970,108.101,108.099,108.281,107.874,108.069,108.052,108.232,...,115.474,114.813,115.310,115.281,115.791,115.229,115.662,115.942,115.551,115.845
13189,108.099,108.281,107.874,108.069,108.052,108.232,107.814,107.962,107.962,108.145,...,115.791,115.229,115.662,115.633,115.942,115.551,115.845,116.198,115.797,116.130
13190,108.052,108.232,107.814,107.962,107.962,108.145,107.477,107.875,107.887,108.197,...,115.942,115.551,115.845,115.818,116.198,115.797,116.130,117.357,116.069,117.293


In [25]:
#predict high
label_h = 'p_High_next_period1' 
print("Summary of class variable: \n", train_close[label_h].describe())

Summary of class variable: 
 count    12966.000000
mean       153.114249
std         66.742531
min         75.989000
25%        108.060000
50%        120.710000
75%        208.650000
max        315.210000
Name: p_High_next_period1, dtype: float64


In [26]:
#predict low
label_l = 'p_Low_next_period1' 
print("Summary of class variable: \n", train_close[label_l].describe())

Summary of class variable: 
 count    12966.000000
mean       152.353531
std         67.166653
min         75.570000
25%        107.170000
50%        119.621500
75%        208.650000
max        315.210000
Name: p_Low_next_period1, dtype: float64


In [27]:
#predict close
label_c = 'p_Close_next_period1' 
print("Summary of class variable: \n", train_close[label_c].describe())

Summary of class variable: 
 count    12966.000000
mean       152.745800
std         66.946244
min         75.783000
25%        107.630000
50%        120.175000
75%        208.650000
max        315.210000
Name: p_Close_next_period1, dtype: float64


In [28]:
train_close = pd.DataFrame(train_close.drop_duplicates(keep=False))

In [29]:
train_close.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12966 entries, 230 to 13195
Columns: 927 entries, p_Open_last_period230 to p_Close_next_period1
dtypes: float64(927)
memory usage: 91.8 MB


In [30]:
#predict high
save_path='agModels-predictHigh'
predictor_h = TabularPredictor(label=label_h, path=save_path)
predictor_h.fit(train_close, hyperparameters={ 'XGB':{} } , presets='optimize_for_deployment')
#predictor.fit(train_close, excluded_model_types=['CatBoost'], presets='optimize_for_deployment') #is slower

Presets specified: ['optimize_for_deployment']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictHigh/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    12966
Train Data Columns: 926
Label Column: p_High_next_period1
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (315.21, 75.989, 153.11425, 66.74253)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitt

In [ ]:
#predict low
save_path='agModels-predictLow'
predictor_l = TabularPredictor(label=label_l, path=save_path)
predictor_l.fit(train_close, hyperparameters={ 'XGB':{} } , presets='optimize_for_deployment')
#predictor.fit(train_close, excluded_model_types=['CatBoost'], presets='optimize_for_deployment') #is slower

Presets specified: ['optimize_for_deployment']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictLow/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    12966
Train Data Columns: 926
Label Column: p_Low_next_period1
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (315.21, 75.57, 152.35353, 67.16665)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting

In [ ]:
#predict close
save_path='agModels-predictClose'
predictor_c = TabularPredictor(label=label_c, path=save_path)
predictor_c.fit(train_close, hyperparameters={ 'XGB':{} } , presets='optimize_for_deployment')
#predictor.fit(train_close, excluded_model_types=['CatBoost'], presets='optimize_for_deployment') #is slower

In [ ]:
#contrast leaderboard result of train and test data
#predictor.leaderboard(train_close, silent=True)

In [ ]:
#predict high
test_close = TabularDataset(df_test_close)
y_test_h = test_close[label_h]
test_close_nolab_h = test_close.drop(columns=[label_h])
test_close_nolab_h.tail()

In [ ]:
#predict low
test_close = TabularDataset(df_test_close)
y_test_l = test_close[label_l]
test_close_nolab_l = test_close.drop(columns=[label_l])
test_close_nolab_l.tail()

In [ ]:
#predict close
test_close = TabularDataset(df_test_close)
y_test_c = test_close[label_c]
test_close_nolab_c = test_close.drop(columns=[label_c])
test_close_nolab_c.tail()

In [ ]:
#only the last observation in the list is the prediction, the other observations are past.
#predict p_High_next_period1
predictor_h = TabularPredictor.load('agModels-predictHigh')
y_pred_h = predictor_h.predict(test_close_nolab_h)
print("Predictions for High: \n", y_pred_h)
perf_h = predictor_h.evaluate_predictions(y_true = y_test_h, y_pred = y_pred_h, auxiliary_metrics=True)

In [ ]:
#only the last observation in the list is the prediction, the other observations are past.
#predict p_Low_next_period1
predictor_l = TabularPredictor.load('agModels-predictLow')
y_pred_l = predictor_l.predict(test_close_nolab_l)
print("Predictions for Low: \n", y_pred_l)
perf_l = predictor_l.evaluate_predictions(y_true = y_test_l, y_pred = y_pred_l, auxiliary_metrics=True)

In [ ]:
#only the last observation in the list is the prediction, the other observations are past.
#predict p_Close_next_period1
predictor_c = TabularPredictor.load('agModels-predictClose')
y_pred_c = predictor_c.predict(test_close_nolab_c)
print("Predictions for Close: \n", y_pred_c)
perf_c = predictor_c.evaluate_predictions(y_true = y_test_c, y_pred = y_pred_c, auxiliary_metrics=True)

In [ ]:
#put predicted where it will be actual
d = {'pred_h':y_pred_h,'pred_l':y_pred_l,'pred_c':y_pred_c}
df_t = pd.DataFrame(data=d)
#df_t['test']=df_t['test']
#.shift(-1)
df_t

In [ ]:
# f=plt.figure()
# f.set_figwidth(10)
# f.set_figheight(10)
# line1 = plt.plot(df_t.test, label = 'actual close', marker='x', color='grey')
# #include actual high and low range

# line2 = plt.plot(df_t.pred, label = 'predicted close', marker='x', color='g')
# plt.legend()
# plt.show()
 

In [ ]:
#include actual high and low range
df_t['test_high']=df_raw1.iloc[-40:,1:2].shift(-1)
df_t['test_low']=df_raw1.iloc[-40:,2:3].shift(-1)
df_t['test_close']=df_raw1.iloc[-40:,3:4].shift(-1)
df_t

In [ ]:
f=plt.figure()
f.set_figwidth(10)
f.set_figheight(10)
#include actual high and low range
line1_h = plt.plot(df_t.test_high, label = 'actual high', marker='x', color='r')
line1_l = plt.plot(df_t.test_low, label = 'actual low', marker='x', color='r')
line1_c = plt.plot(df_t.test_close, label = 'actual close', marker='x', color='r')

#overlay predicted high and low and close
line2_ph = plt.plot(df_t.pred_h, label = 'predicted high', marker='x', color='g')
line2_pl = plt.plot(df_t.pred_l, label = 'predicted low', marker='x', color='g')
line2_pc = plt.plot(df_t.pred_c, label = 'predicted close', marker='x', color='g')


plt.legend()
plt.show()
 

In [ ]:
print(stop_for_sell)
print(stop_for_buy)

In [ ]:
# clears the model after a run so you can contrast new runs.
# predictor.unpersist_models('ag_predict_interval')
# predictor.save_space()
# predictor.delete_models(models_to_keep='best', dry_run=False)
%rm -rf agModels-predictHigh 
%rm -rf agModels-predictLow
%rm -rf agModels-predictClose
!ls

